# Is there a difference between male and female brains?
The topic of whether or not there is a typical "male" and "female" brain is hotly contested. Several studies, including [this one]() and [this one]() have used a mosaic of various brain features to try and answer this question, with conflicting results.

One of the problems that arise from previous mosaic analysis is that they use the absolute volume of various features of the brain without taking into account that the mean size of a male person's brain is larger than a female person's brain because on average males are larger than females.

In this analysis, we explore if male and female brains can be categorized using volumes of various brain regions expressed as percentages in order to remove the effect of total volume.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Explore the Data